在 chat 中维护上下文

简单的讲所有聊天记录都传给 llm 很容易受到 llm 的上下文窗口限制，也会消耗大量 token  
并且用户后续发送的信息可能与前面聊天讨论的话题完全无关，可能会影响回答的质量

因此我们可以在聊天记录进行一些处理，这就是 memory

In [ ]:
import { ChatMessageHistory } from "langchain/stores/message/in_memory";
import { HumanMessage, AIMessage } from "@langchain/core/messages";

const history = new ChatMessageHistory();

// 添加 Message 历史信息
await history.addMessage(new HumanMessage('hi'));
await history.addMessage(new AIMessage("What can I do for you?"))

const message = await history.getMessages();

console.log(message);

In [ ]:
import { load } from "dotenv"
const env = await load({
  envPath: ".env.local",
})

const process = { env }

const chatOptions = {
  openAIApiKey: process.env.Tongyi_API_KEY,
  temperature: 1.5,
  model: "deepseek-chat",
  modelName: "deepseek-chat",
  configuration: {
    baseURL: process.env.BASE_URL,
  },
  azureOpenAIBasePath: process.env.BASE_URL,
}
console.log(chatOptions)


In [ ]:
import { ChatPromptTemplate, MessagesPlaceholder } from "@langchain/core/prompts";
import { ChatOpenAI } from "@langchain/openai"

const chatModel = new ChatOpenAI(chatOptions)
const prompt = ChatPromptTemplate.fromMessages([
  [
    "system",
    `You are a helpful assistant. Answer all questions to the best of your ability.
    You are talkative and provides lots of specific details from its context. 
    If the you does not know the answer to a question, it truthfully says you do not know.`,
  ],
  new MessagesPlaceholder("history_message"),
])

const chain = prompt.pipe(chatModel);
console.log("🚀 ~ chatModel:", chatModel)


In [ ]:
import { ChatMessageHistory } from "langchain/stores/message/in_memory"
import { HumanMessage, AIMessage } from "@langchain/core/messages"

const history = new ChatMessageHistory()
await history.addMessage(new HumanMessage("hi, my name is Ginlon"))

const res1 = await chain.invoke({
  history_message: await history.getMessages(),
})
console.log(res1)